# <center>  Решение тестового задания при помощи модели SBERT и NLTK</center>

Задача: найти фрагмент в тексте по заданному запросу. Для решения используется алгоритм, используюий подход поисковой системы. Вместо документов - отдельные предложения. Создаются эмбединги запроса и предложений, а модель возвращает предложение, наиболее похожее на запрос.

**Предобработка текста**

Пусть искомый фрагмент - предложение из текста. В таком случае требуется разделить текст на предложения. Проблема заключается в том, что некоторые предложения не разделены знаком припинания. Для решения этой проблемы используются регулярные выражения и библиотека NLTK. В будущем для улучшения модели можно создать отдельную модель, которая бы расставляла недостающие знаки препинания

**Получения эмбедингов**

Для создания эмбедингов предложений используется модель [SBERT](https://arxiv.org/pdf/1908.10084.pdf). Модель принимает на вход предложения и возвращает эмбединги предложений.

**Тюнинг модели**

На основе прогноза модель дообучается. При помощи косинусной меры определяется похожесть предложений. Наиболее похожее предложение сохраняется в качестве прогноза. Если прогноз совпал с ответом, то моделе сообщается, что найденный вектор и запрос схожи, если нет - то сообщается что у них низкая схожесть. Для определения схожести используется косинусная мера, для определения схожести ответа и прогноза - квадрат растояния Джаро (т.к. требуется понять, сколько символов совпало и не сильно штрафовать за промах в несколько символов)

## Установка модулей

In [11]:
!pip install -qU transformers sentence-transformers

In [12]:
!pip install nltk

## Импорт библиотек и данных

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as pls
%matplotlib inline

In [4]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
import json
import os
import tqdm
import re

In [51]:
import torch

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

from sklearn.metrics.pairwise import cosine_similarity
from jellyfish import jaro_distance

In [52]:
with open('train.json', 'r', encoding='utf-8') as f:
    train_raw = json.loads(f.read())

train = pd.json_normalize(train_raw)
train.columns = train.columns.str.replace('.', '_')
train['text'] = train['text'].astype(str)
train['label'] = train['label'].astype(str)
train['extracted_part_text'] = train['extracted_part_text'].str[0]
train['extracted_part_answer_start'] = train['extracted_part_answer_start'].str[0].astype('int')
train['extracted_part_answer_end'] = train['extracted_part_answer_end'].str[0].astype('int')

train.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_13380\4100176283.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train.columns = train.columns.str.replace('.', '_')


,id,text,label,extracted_part_text,extracted_part_answer_start,extracted_part_answer_end
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1279,1343
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1222,1318
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1297,1343
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1304,1350
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1302,1348


In [ ]:
train.info()

In [15]:
with open('test.json', 'r', encoding='utf-8') as f:
    test_raw = json.loads(f.read())

test = pd.json_normalize(test_raw)
test.columns = test.columns.str.replace('.', '_')
test['text'] = test['text'].astype(str)
test['label'] = test['label'].astype(str)
test.head()

C:\Users\danie\AppData\Local\Temp\ipykernel_13380\2864110668.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test.columns = test.columns.str.replace('.', '_')


,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      318 non-null    int64 
 1   text    318 non-null    object
 2   label   318 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.6+ KB


## Загрузка модели

In [53]:
if os.path.exists('model/'):
    model = SentenceTransformer('model/')
else:
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')

## Вспомогательные функции

Предобработка текста

In [25]:
def tokenize(text: str):
    """
    Функция для токенизации текста. Сначала функция доставляет недостающие точки
    при помощи шаблона, а затем токенизирует его.
    """
    pattern = "(?<=[а-яё])\s+(?=[А-ЯЁ][^А-ЯЁ])"
    # Шаблон - слово с маленькой буквы, пробел, слово с большой буквы
    text_sub = re.sub(pattern, '. ', text) 
    sents = sent_tokenize(text_sub, language='russian')
    # Избавляемся от лишних точек на конце
    return [sent.replace('. ', '') if sent[-1] != '.' else sent[:-1].replace('. ', '') for sent in sents]

In [ ]:
expected_text = []
expected_start = []
expected_end = []

for i in tqdm.tqdm(range(test.shape[0])):
    text = test.text[i]
    label = test.label[i]
    pred_token, pred_start, pred_end = predict(model, text, label)

    expected_text.append(pred_token)
    expected_start.append(pred_start)
    expected_end.append(pred_end) 
    

 14%|█▍        | 46/318 [03:00<17:46,  3.92s/it]


KeyboardInterrupt: ignored

In [38]:
def predict(model, text: str, label: str, fit=False) -> [str, int, int]:
    """
    Cоздание прогноза по входным данным. Возвращает фразу, начало и конец фразы в исходном тексте
    model: модель SBERT
    text: текст для поиска
    label: фраза для поиска
    """
    # Проверка на вхождение label в текст
    if label not in text:
        return '', 0, 0

    # Токенизация
    tokens = tokenize(text)


    # Получаем эмбединг лейбла и эмбединги токенов
    embedding_label = model.encode(label)
    embeddings_tokens = model.encode(tokens)

    # Считаем "похожесть" эмбедингов при помощи косинусной метрики
    similarity = cosine_similarity(embedding_label.reshape(1, -1), 
                                 embeddings_tokens)[0]

    # Сохраняем предсказанный токен и ищем его начало
    predicted_token = tokens[np.argmax(similarity)]
    predicted_start = text.find(predicted_token)


    # Возвращаем предсказанный токен и его позицию в тексте
    return predicted_token, predicted_start, predicted_start + len(predicted_token)

In [89]:
def fit(dataset: pd.DataFrame, model, n_epochs=5, batch_size=64, dataloader_batch_size=16):
    """"""
    train_examples = []
    for epoch in range(n_epochs):
        for i in tqdm.tqdm(range(dataset.shape[0]), postfix=f'epoch №{epoch}'):
            train_text = dataset.text[i]
            train_label = dataset.label[i]
            # Предсказывает токен
            pred, start, end = predict(model, train_text, train_label)
            
            # Используется квадрат расстояния Джаро между предсказанным и ожидаемым текстом
            # sim = 1 - полное совпадение, sim = 0 - фразы полностью не совпадают
            # sim устанавливается в качестве меры схожести между запросом и полученным ответом
            sim = jaro_distance(pred, dataset.extracted_part_text[i])
            train_examples.append(InputExample(texts=[train.label[i], pred], label=sim**2))
        if i % batch_size == 0 and i > 0:
            # тюнинг модели после накопления данных
            train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=dataloader_batch_size)
            train_loss = losses.CosineSimilarityLoss(model)
            model.fit(train_objectives=[(train_dataloader, train_loss)], 
                      epochs=1, 
                      warmup_steps=100,
                      output_path='model/')
            train_examples = []

## Обучение модели

In [90]:
# Обучение модели по датасету без пропусков
dataset = train[(train.extracted_part_answer_start != 0) & (train.extracted_part_answer_end != 0)].reset_index(drop=True)
fit(dataset, model)

 31%|███       | 457/1492 [44:48<1:41:29,  5.88s/it, epoch №0]


KeyboardInterrupt: 

In [91]:
model.save('model/')

## Создание прогноза для тестовой выборки

In [92]:
expected_text = []
expected_start = []
expected_end = []

for i in tqdm.tqdm(range(test.shape[0])):
    text = test.text[i]
    label = test.label[i]
    pred_token, pred_start, pred_end = predict(model, text, label)

    expected_text.append(pred_token)
    expected_start.append(pred_start)
    expected_end.append(pred_end) 

100%|██████████| 318/318 [23:48<00:00,  4.49s/it]


In [94]:
test['extracted_part_text'] = expected_text
test['extracted_part_answer_start'] = expected_start
test['extracted_part_answer_end'] = expected_end

In [95]:
test

,id,text,label,extracted_part_text,extracted_part_answer_start,extracted_part_answer_end
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,,0,0
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Обеспечение исполнения контракта,1215,1247
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,Обеспечение исполнения настоящего,1679,1712
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,Обеспечение исполнения настоящего,1691,1724
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,Обеспечение исполнения настоящего,1693,1726
...,...,...,...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств,,0,0
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств,Возврат обеспечения гарантийных обязательств п...,2372,2425
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств,,0,0
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств,,0,0


In [97]:
test.to_json('test_ans.json', orient="records", encoding=)